In [2]:
## Check optimal throughput IS achieved when dispatching 100% based on service time reduction/affinity
import sys
import os

module_path = os.path.abspath(os.path.join('../..'))
#print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
#print(sys.path)

from parallel import Invoker

# Calculate params for simulations
from numpy import arange,linspace,logspace
import math

NUM_SIMULATED_CORES = 16
STATIC_NUM_FUNCTIONS = 16
CACHE_SIZE=64
LIB_FRACTION = 0.25
F_STEP = 2
FunctionRange = [16]
workset_range = [40]
GroupingRange = logspace(base=2,start=0,stop=4,num=5)

def divisorGenerator(n):
    large_divisors = []
    for i in range(1, int(math.sqrt(n) + 1)):
        if n % i == 0:
            yield i
            if i*i != n:
                large_divisors.append(n / i)
    for divisor in reversed(large_divisors):
        yield divisor
            
def filter_configs_by_cores(num_funcs,grouping,divisor_list):
    num_queues = num_funcs / grouping
    if num_queues in divisor_list:
        return True
        return (num_funcs,grouping,cores_per_q)
    
# Cores_per_q is always even for now by virtue of filter_configs_by_cores
def setup_final_config(num_funcs,grouping,workset,affinity=False,ideal=False):
    num_queues = num_funcs / grouping
    cores_per_q = NUM_SIMULATED_CORES / num_queues
    return {'funcs': int(num_funcs), 'grouping' : int(grouping), 'workset' : workset,
            'cores_per_q': int(cores_per_q), 'affinity' : affinity, 'ideal' : ideal}
    
# Find best grouping model for a given working set, assuming BTB is contention
def best_grouping_btb(WorkingSet,core_divisors):
    unique_working_set = math.floor(WorkingSet * 1024 * (1-LIB_FRACTION))
    num_insts = unique_working_set/4
    num_branches = num_insts/5
    funcs_per_btb = math.floor(6*1024 / float(num_branches))
    
    largest_group = 1
    for x in core_divisors:
        if x > largest_group and x <= funcs_per_btb:
            largest_group = x
    
    return largest_group

# Find best grouping model for a given working set, assuming cache is contention
def best_grouping_cache(WorkingSet,core_divisors):
    unique_working_set = math.floor(WorkingSet * 1024 * (1-LIB_FRACTION))
    funcs_per_cache = math.floor((1024 * CACHE_SIZE) / float(unique_working_set))
    
    largest_group = 1
    for x in core_divisors:
        if x > largest_group and x <= funcs_per_cache:
            largest_group = x
    
    return largest_group

# Find the best grouping model for each working set being simulated, and create a param dict for it.

    
# Add affinity solution to parameter list 
affinity_params = list(map(lambda x : setup_final_config(STATIC_NUM_FUNCTIONS,1,x,affinity=True),workset_range))
    # IS THIS RIGHT? shouldn't affinity be 1xN?
final_params.extend(affinity_params)

# Add theoretical best 99th% solution to list of params
# - 1 queue, 16 workers, ideal service time
#ideal_args = list(map(lambda x : setup_final_config(STATIC_NUM_FUNCTIONS,STATIC_NUM_FUNCTIONS,
#                                                    x,affinity=False,ideal=True),workset_range)) 
    # 0 working set = ideal serv. time
#print([ideal_args])
#final_params.extend(ideal_args)

# Make config for FIFO - 1x16

# Make config for FIFO - (GF = best)
core_divisors = list(divisorGenerator(NUM_SIMULATED_CORES))
best_groupings = [(best_grouping_cache(w,core_divisors),w) for w in workset_range]
final_params = list(map(lambda x : setup_final_config(STATIC_NUM_FUNCTIONS,x[0],x[1]),best_groupings))
print(final_params)

# Make config for SST - 1x16

TypeError: required field "type_ignores" missing from Module